<a href="https://colab.research.google.com/github/david-meltzer/LLMs/blob/main/distilgpt2_simplewiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/gpt2

/content/drive/MyDrive/gpt2


In [3]:
!ls

gpt2_finetune.ipynb


In [6]:
!pip install transformers[torch]
!pip install datasets
!pip install apache_beam
!pip install mwparserfromhell

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.0/152.0 kB 23.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 85.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.6/603.6 kB 31.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 75.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.3/300.3 kB 38.0 MB/s eta 0:00:00
  Created wheel for crcmod: filename=crcmod-1.7-cp310-cp310-linux_x86_64.whl size=31407 sha256=1c7ce5f43b61939b85ad1477876c8cf894c9a3cb1c5df7215a1a7195b91a8242
  Stored in directory: /root/.cache/pip/wheels/85/4c/07/72215c529bd59d67e

In [7]:
from pprint import pprint as pp

In [8]:
from datasets import load_dataset
wiki = load_dataset("wikipedia", "20220301.simple", split = 'train')
#wiki_small = wiki.select(range(5000))
#wiki_small.save_to_disk('wiki_small')

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/235M [00:00<?, ?B/s]

In [11]:
wiki.save_to_disk('wiki_simple')

Saving the dataset (0/1 shards):   0%|          | 0/205328 [00:00<?, ? examples/s]

In [12]:
wiki_split = wiki.train_test_split(test_size = 5000)
wiki_split

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 200328
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 5000
    })
})

In [13]:
wiki_split['train'][4]

{'id': '715652',
 'url': 'https://simple.wikipedia.org/wiki/Eric%20Pleskow',
 'title': 'Eric Pleskow',
 'text': "Eric Pleskow (April 24, 1924 – October 1, 2019) was an Austrian-born American movie producer. He was President of the movie studios United Artists and Orion Pictures. He was born in Vienna.\n\nUnder his presidency at United Artists, the company won the Academy Award for Best Picture three years in a row: One Flew Over the Cuckoo's Nest (1975), Rocky (1976) and Annie Hall (1977).\n\nIn 1978, Pleskow left United Artists and built up Orion Pictures, which he ran until 1992. Their greatest successes under his management were Amadeus, Dances with Wolves and The Silence of the Lambs.\n\nFrom 1998 until his death, he was the President of the Vienna International Film Festival.\n\nPleskow died on October 1, 2019 in Westport, Connecticut at the age of 95.\n\nReferences\n\nOther websites\n \n\n1924 births\n2019 deaths\nNaturalized citizens of the United States\nAustrian movie producer

In [28]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [29]:
tokenizer

GPT2TokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True)

In [16]:
pp(tokenizer.model_max_length)
pp(tokenizer.is_fast)

1024
True


In [17]:
context_len = 512

outputs = tokenizer(
    wiki_split['train'][:50]['text'],
    truncation=True,
    max_length=context_len,
    return_overflowing_tokens=True,
    return_length=True
)

In [18]:
outputs.keys()

dict_keys(['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping'])

In [19]:
pp(f'Number of input_ids: { len( outputs["input_ids"] ) }')
pp(outputs['length'], compact = True, width = 150)

'Number of input_ids: 65'
[512, 11, 153, 138, 165, 222, 82, 114, 96, 318, 36, 512, 216, 150, 87, 143, 191, 268, 405, 167, 512, 25, 512, 110, 51, 381, 80, 70, 45, 49, 154, 128,
 394, 115, 133, 510, 19, 108, 119, 453, 166, 25, 30, 378, 41, 79, 40, 158, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 512, 20, 123, 250, 148,
 184, 41]


In [20]:
pp(outputs['overflow_to_sample_mapping'], compact = True, width = 150)

[0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 11, 12, 13, 14, 15, 16, 17, 18, 18, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
 36, 37, 38, 39, 40, 41, 42, 43, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 44, 45, 46, 47, 48, 49]


In [31]:
context_len = 512
def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_len,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_len:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


wiki_tokenized = wiki_split.map(
    tokenize, batched = True, num_proc = 4, remove_columns=wiki_split["train"].column_names
)

wiki_tokenized

Map (num_proc=4):   0%|          | 0/200328 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 42648
    })
    test: Dataset({
        features: ['input_ids'],
        num_rows: 1137
    })
})

#Model

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [36]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

#Train

In [33]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(
    output_dir="distilgpt2_simplewiki",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
#    per_device_train_batch_size=8,  # default batch size per device during training, batch size 16 doesn't fit on T4 GPU
#    per_device_eval_batch_size=8,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=wiki_tokenized["train"],
    eval_dataset=wiki_tokenized["test"],
    data_collator=data_collator,
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,3.123900,3.028715
